# Extracting results from clustered FedAvg

In [36]:
import json
import re
import pandas as pd
from pathlib import Path
from functools import reduce

benign = (None, None)
targeted = {}
untargeted = {}

run_dirs = [file_or_dir for file_or_dir in Path('').iterdir() if file_or_dir.is_dir()]

def get_acc_miss(metrics: dict) -> tuple[float, float]:
    acc, miss, n = 0, 0, 0
    for client_name, client_metrics in metrics.items():
        m = client_metrics[-1][1]
        if "client" in client_name:
            acc += m["accuracy"]
            miss += m["missrate"]
            n += 1
    return acc / n, miss / n

def get_miss_target(metrics: dict, target: str) -> float:
    miss, n = 0, 0
    for client_name, client_metrics in metrics.items():
        m = client_metrics[-1][1]
        if "client" in client_name:
            stats = next(d for d in m["attack_stats"] if d["attack"].lower().replace(" ", "") == target.lower().replace(" ", ""))
            miss += stats["missed"] / stats["count"]
            n += 1
    return miss / n

## Benign runs

In [37]:
benign_runs = [rdir for rdir in run_dirs if "benign" in rdir.name]
accs_l = []
miss_l = []
for run in benign_runs:
    acc, miss = get_acc_miss(json.load(open(run / "metrics.json")))
    accs_l.append(acc)
    miss_l.append(miss)

benign_accuracy = sum(accs_l) / len(accs_l)
benign_missrate = sum(miss_l) / len(miss_l)

benign_accuracy, benign_missrate

(0.9901148463994899, 0.01813954127551561)

## Untargeted attacks

Note: in untargeted attacks, the *attack success rate* is the percentage of samples that where misclassified after the poisoning attack. Also, attacks are only perpetrated on "Bot-IoT", so we reuse the results from the benign runs for the other datasets, and merge them with the results from the attacks on "Bot-IoT".

In [38]:
benign_runs_wo_botiot = [rdir for rdir in benign_runs if "botiot" not in rdir.name]
accs_l_no_botiot = []
miss_l_no_botiot = []
for run in benign_runs:
    acc, miss = get_acc_miss(json.load(open(run / "metrics.json")))
    accs_l_no_botiot.append(acc)
    miss_l_no_botiot.append(miss)

benign_accuracy_no_botiot = sum(accs_l_no_botiot) / len(accs_l_no_botiot)
benign_missrate_no_botiot = sum(miss_l_no_botiot) / len(miss_l_no_botiot)

In [39]:
untargeted_runs = [rdir for rdir in run_dirs if "untargeted" in rdir.name and "botiot" in rdir.name] 
print(f"Found {len(untargeted_runs)} runs")
run_re = re.compile(r".*stealth(?P<noise>\d\.\d).*distribution=(?P<dist>\w+).*")

acc_df = pd.DataFrame(
    columns=["10", "20", "30", "40", "50", "60", "70", "80", "90", "100"],
    index=["benign", "lone", "sybils_min", "sybils_maj"]
)
asr_df = acc_df.copy()

for run in untargeted_runs:
    m = run_re.match(run.name)
    if m is None:
        print(f"Error: no match for `{run.name}`")
        break
    dist = m.group("dist").split("_")
    dataset = dist.pop()
    scenario = "_".join(dist)
    noise = str(int(float(m.group("noise"))*100))
    metrics: dict = json.load(open(run / "metrics.json"))
    acc, _ = get_acc_miss(metrics)
    acc_df.loc[scenario, noise] = acc
    asr_df.loc[scenario, noise] = 1 - acc

# At this point, we have the accuracy and ASR for each scenario and noise level, and the
# benign accuracy for the `stealth1.0` scenario, even though there are no attackers. The
# results are fot the Bot-IoT dataset ONLY.

acc_df.loc["benign"] = acc_df.loc["benign", "100"] # The benign accuracy is the same for all noise levels

asr_df = 1 - acc_df # ASR = 1 - accuracy

# `benign_accuracy_no_botiot` is the averaged accuracy without attacks of all the other
# (3) datasets.
acc_df += benign_accuracy_no_botiot * 3 
acc_df /= 4 # Average accuracy including botiot partitipants



Found 32 runs
Error: no match for `+scenario_target=untargeted,archi=fedavg,fl.drop_class=False,scenario_distribution=_single_cluster_sybils_min_botiot,xp.seed=56`


In [40]:
acc_df

,10,20,30,40,50,60,70,80,90,100
benign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lone,0.992336,0.992334,NaN,NaN,0.992471,0.992324,0.99233,NaN,NaN,NaN
sybils_min,0.992334,NaN,0.992322,NaN,0.992289,0.992287,NaN,NaN,0.9816,0.976939
sybils_maj,NaN,NaN,NaN,NaN,0.992244,0.992141,NaN,NaN,NaN,NaN


In [41]:
asr_df

,10,20,30,40,50,60,70,80,90,100
benign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lone,0.001,0.00101,NaN,NaN,0.00046,0.00105,0.001025,NaN,NaN,NaN
sybils_min,0.00101,NaN,0.001055,NaN,0.00119,0.001195,NaN,NaN,0.043945,0.06259
sybils_maj,NaN,NaN,NaN,NaN,0.00137,0.00178,NaN,NaN,NaN,NaN


## Targeted attacks

Note: in targeted attacks, the *attack success rate* is the missrate of the model on the targeted class after the poisoning attack.

In [42]:

targeted_runs = [rdir for rdir in run_dirs if ("untargeted" not in rdir.name or "benign" in rdir.name) and "botiot" in rdir.name] # and "benign" not in rdir.name 
print(f"Found {len(untargeted_runs)} runs")
run_re = re.compile(r".*stealth(?P<noise>\d\.\d).*distribution=(?P<dist>\w+).*")

acc_df = pd.DataFrame(
    columns=["10", "20", "30", "40", "50", "60", "70", "80", "90", "100"],
    index=["benign", "lone", "sybils_min", "sybils_maj"]
)
asr_df = acc_df.copy()

for run in targeted_runs:
    m = run_re.match(run.name)
    if m is None:
        print(f"Error: no match for `{run.name}`")
        break
    dist = m.group("dist").split("_")
    dataset = dist.pop()
    scenario = "_".join(dist)
    noise = str(int(float(m.group("noise"))*100))
    metrics: dict = json.load(open(run / "metrics.json"))
    
    acc, _ = get_acc_miss(metrics)
    acc_df.loc[scenario, noise] = acc

    miss = get_miss_target(metrics, "reconnaissance")
    asr_df.loc[scenario, noise] = miss

# At this point, we have the accuracy and ASR for each scenario and noise level, and the
# benign accuracy for the `stealth1.0` scenario, even though there are no attackers. The
# results are fot the Bot-IoT dataset ONLY.

acc_df.loc["benign"] = acc_df.loc["benign", "100"] # The benign accuracy is the same for all noise levels

# `benign_accuracy_no_botiot` is the averaged accuracy without attacks of all the other
# (3) datasets.
acc_df += benign_accuracy_no_botiot * 3 
acc_df /= 4 # Average accuracy including botiot partitipants


Found 32 runs


In [43]:
acc_df

,10,20,30,40,50,60,70,80,90,100
benign,0.99235,0.99235,0.99235,0.99235,0.99235,0.99235,0.99235,0.99235,0.99235,0.99235
lone,0.992337,0.992349,0.992334,0.992332,0.992334,0.992311,0.992332,0.992317,0.992302,0.992301
sybils_min,0.992345,0.992334,0.992315,0.992316,0.992294,0.992286,0.992279,0.992251,0.992237,0.983336
sybils_maj,0.992331,0.992327,0.992311,0.992311,0.992289,0.99225,0.992231,0.992034,0.983257,0.982286


In [44]:
asr_df

,10,20,30,40,50,60,70,80,90,100
benign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00058
lone,0.000942,0.000725,0.00116,0.00116,0.001087,0.002392,0.001232,0.001957,0.003697,0.004495
sybils_min,0.000797,0.001522,0.00203,0.002102,0.004205,0.004132,0.005002,0.006597,0.008192,0.532985
sybils_maj,0.001595,0.001595,0.00232,0.002465,0.00435,0.006597,0.007902,0.019066,0.537045,0.593664
